# PUSHSHIFT

In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

In [2]:
def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?&size=1000&after=' \
          + str(after) + '&before=' + str(before) + '&subreddit=' + str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [3]:
def collectSubData(subm):
    subData = list()

    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])
    numComms = subm['num_comments']
    permalink = subm['permalink']

    subData.append((sub_id, title, url, author, score, created, numComms, permalink, flair))
    subStats[sub_id] = subData

In [4]:
# Subreddit to query
sub = 'opiates'
before = 1622520000  # later date
after = 1619841600  # earlier date
subCount = 0
subStats = {}

In [5]:
data = getPushshiftData(after, before, sub)

while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # print(len(data))
    # print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before, sub)

# print(len(data))
# print(str(len(subStats)) + " submissions")

https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1619841600&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620007243&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620143629&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620265955&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620393878&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620507323&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620622568&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620765629&before=1622520000&subreddit=opiates
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1620867010&b

In [6]:
def updateSubs_file():
    upload_count = 0
    filename = input()
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file:
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID", "Title", "Url", "Author", "Score",
                   "Publish Date", "No. of Comments", "Permalink", "Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1

        print(str(upload_count)+" submissions have been uploaded")
        
updateSubs_file()

pushshift
2076 submissions have been uploaded


# PRAW

In [7]:
import praw
from praw.models import MoreComments
import pandas as pd
import csv

In [8]:
pushshift = pd.read_csv('pushshift')
pushshift = pushshift.reset_index()
len(pushshift['Post ID'])

2076

In [10]:
reddit = praw.Reddit(client_id='uwd2yu4XRqn2hg', 
                     client_secret='8Xp3w6by0qpSAvx7wspls1XbYrcQGQ', 
                     user_agent='Capstone')

posts = []

for i in range(len(pushshift['Post ID'])):
    post = reddit.submission(id=pushshift['Post ID'][i])
    if (post.selftext=='[deleted]') | (post.selftext=='[removed]'):
        continue
    posts.append([post.title, post.author, post.score, post.id, post.subreddit,
                  post.url, post.num_comments, post.selftext, post.created])

In [11]:
posts = pd.DataFrame(posts, columns=['title', 'name', 'score', 'id', 'subreddit',
                                     'url', 'num_comments', 'body', 'created'])